In [5]:
import pandas as pd
import numpy as np
import torch
from paths import *
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
model_mame = "Qwen/Qwen3-4B-Instruct-2507"
# gguf_file="Qwen3-4B-Instruct-2507-UD-Q4_K_XL.gguf"

In [7]:
df = pd.read_excel(f'{DATA_FOLDER}/DocProperties/DocProperties_filled.xlsx')
df

,Name,Expression,Description
0,750InsPremAutocivilka,"__Result = ""---"";\nWorkflow.DTO.Models.Program...",Код знаходить програму з кодом типу страхуванн...
1,750InsPremAsist,"__Result = ""---"";\nWorkflow.DTO.Models.Program...",Код обчислює загальну суму запланованих платеж...
2,704LandAddress,"__Result = """";\nWorkflow.DTO.Models.Program pr...",Код формує текстову адресу земельної ділянки з...
3,739ParamSport,"__Result = ""Ні"";\nWorkflow.DTO.Models.Program ...","Код перевіряє, чи в програмі GetProgramMed вст..."
4,769Rate249,"__Result = ""---"";\nWorkflow.DTO.Models.Program...",Код перевіряє кількість об'єктів страхування в...
...,...,...,...
947,MedInsuranceSumDollar,Workflow.DTO.Models.Program prog = GetProgramM...,NaN
948,MedInsuranceSumEURO,Workflow.DTO.Models.Program prog = GetProgramM...,NaN
949,BranchNameFromParameter,Workflow.DTO.Models.ProgramParameterValue para...,NaN
950,DirectionCode,"__Result = ""---"";\nWorkflow.DTO.Models.Branch ...",NaN


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_mame,
    device_map="auto",
    torch_dtype="auto",
    cache_dir=f'{ML_FOLDER}/Qwen3-4B-Instruct-2507-GGUF'
)

tokenizer = AutoTokenizer.from_pretrained(model_mame, cache_dir=f'{ML_FOLDER}/Qwen3-4B-Instruct-2507-GGUF')

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

D:\WORK\InCore\PyMarkup4CS\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\WORK\InCore\PyMarkup4CS\data\ML\Qwen3-4B-Instruct-2507-GGUF\models--Qwen--Qwen3-4B-Instruct-2507. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [11]:
def summurizator(code_fragment):
    prompt = f'Сумаризуй цей код C# у формі запиту українською мовою — тільки текст\n\n {code_fragment}'
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=16384,
        temperature=0.7,
        min_p = 0.00,
        do_sample=True,
        top_p=0.8,
        top_k = 20,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

    content = tokenizer.decode(output_ids, skip_special_tokens=True)

    return content

In [12]:
mask = df['Description'].isna() | (df['Description'] == '')
df.loc[mask, 'Description'] = df.loc[mask, 'Expression'].apply(summurizator)

In [14]:
df.to_excel(f'{DATA_FOLDER}/DocProperties/DocProperties_NEW_filled.xlsx')